<a href="https://colab.research.google.com/github/nateraw/huggingface-hub-examples/blob/main/timm_finetune_and_push_to_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install git+https://github.com/rwightman/pytorch-image-models
! pip install huggingface_hub
! git config --global credential.helper store

## Download the training script

Here we download the [official training script](https://github.com/rwightman/pytorch-image-models/blob/master/train.py) from the timm repo.

In [ ]:
! wget -nc https://raw.githubusercontent.com/rwightman/pytorch-image-models/master/train.py

## Train! 🚀

In [ ]:
! python train.py ./data/tfds \
    --dataset tfds/oxford_iiit_pet \
    --val-split test \
    --model resnet50 \
    --epochs 5 \
    -b 256 \
    --amp \
    --num-classes 37 \
    --opt adamw \
    --lr 3e-4 \
    --weight-decay .001 \
    --pretrained \
    -j 2 \
    --dataset-download

## Load the Trained Model

In [10]:
import os
from pathlib import Path

import timm
import yaml


output_dir = Path("output/train")
run_dir = output_dir / sorted(os.listdir(output_dir), reverse=True)[0]

args = yaml.safe_load((run_dir / 'args.yaml').read_text())

model = timm.create_model(
    args.get('model'),
    num_classes=args.get('num_classes'),
    in_chans=3,
    checkpoint_path=run_dir / 'model_best.pth.tar'
)

Define the labels so they can be used in Hugging Face inference API. We'll include them in the `model_config` a couple cells down from here

In [11]:
labels = [
    "Abyssinian", "american_bulldog", "american_pit_bull_terrier",
    "basset_hound", "beagle", "Bengal", "Birman", "Bombay", "boxer",
    "British_Shorthair", "chihuahua", "Egyptian_Mau", "english_cocker_spaniel",
    "english_setter", "german_shorthaired", "great_pyrenees", "havanese",
    "japanese_chin", "keeshond", "leonberger", "Maine_Coon",
    "miniature_pinscher", "newfoundland", "Persian", "pomeranian", "pug",
    "Ragdoll", "Russian_Blue", "saint_bernard", "samoyed", "scottish_terrier",
    "shiba_inu", "Siamese", "Sphynx", "staffordshire_bull_terrier",
    "wheaten_terrier", "yorkshire_terrier"
]

## Push to Hub! 🤗

Before we push, though, we have to authenticate with the Hub.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Finally, we push to the Hub! The repo pushed in the cell below can be seen [here](https://huggingface.co/nateraw/resnet50-oxford-iiit-pet-v2).

In [ ]:
from timm.models.hub import push_to_hf_hub

url = push_to_hf_hub(
    model,
    'resnet50-oxford-iiit-pet-v2',
    model_config={'labels': labels}
)
print(f"Check out the repo we made here: {url}")